# BigCommerce Schema — Data Exploration & Cleaning

This notebook explores the raw BigCommerce schema export (`bigcommerce_raw.csv`), identifies quality issues, and produces a cleaned schema CSV ready for the text2sql retriever pipeline.

**Downstream format required** (`data/schemas/bigcommerce_schema_1.csv`):
| table_name | column_name | data_type |
|---|---|---|

In [47]:
import pandas as pd

In [48]:
df = pd.read_csv("bigcommerce_raw.csv")

df.head()

,table_name,column_name,ordinal_position,is_nullable,data_type
0,applied_coupon,id,1,YES,STRING
1,applied_coupon,display_name,2,YES,STRING
2,applied_coupon,discounted_amount,3,YES,FLOAT64
3,applied_coupon,code,4,YES,STRING
4,applied_coupon,coupon_type,5,YES,INT64


## 1. Basic Info

In [49]:
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nDtypes:\n{df.dtypes}")
print(f"\nNull counts:\n{df.isnull().sum()}")
print(f"\nDuplicate rows: {df.duplicated().sum()}")

Shape: (1615, 5)

Columns: ['table_name', 'column_name', 'ordinal_position', 'is_nullable', 'data_type']

Dtypes:
table_name            str
column_name           str
ordinal_position    int64
is_nullable           str
data_type             str
dtype: object

Null counts:
table_name          0
column_name         0
ordinal_position    0
is_nullable         0
data_type           0
dtype: int64

Duplicate rows: 0


## 2. Table & Column Distribution

In [50]:
table_counts = df["table_name"].value_counts()
print(f"Unique tables: {df['table_name'].nunique()}")
print(f"Unique columns: {df['column_name'].nunique()}")
print(f"Unique data types: {df['data_type'].nunique()}")
print(f"\n--- Columns per table (top 15) ---")
print(table_counts.head(15))
print(f"\n--- Columns per table (bottom 10) ---")
print(table_counts.tail(10))

Unique tables: 145
Unique columns: 669
Unique data types: 7

--- Columns per table (top 15) ---
table_name
orders                    83
product                   68
store                     52
order_product             47
checkout                  43
order_transaction         36
order_shipping_address    33
product_variant_option    33
order_shipment            25
product_variant           25
cart_physical_item        24
consignment               24
catalog_variant           23
cart_digital_item         20
category_tree             19
Name: count, dtype: int64

--- Columns per table (bottom 10) ---
table_name
tax                                         4
channel_app                                 3
consignment_discount                        3
consignment_line_item                       3
customer_channel                            3
product_gift_wrapping_option                3
product_meta_keyword                        3
checkout_billing_address_custom_field       2
consignment_bi

In [51]:
# Data type distribution
print("--- Data type distribution ---")
print(df["data_type"].value_counts())
print(f"\n--- is_nullable distribution ---")
print(df["is_nullable"].value_counts())

--- Data type distribution ---
data_type
STRING       680
INT64        352
BOOL         236
TIMESTAMP    212
FLOAT64      125
JSON           9
DATETIME       1
Name: count, dtype: int64

--- is_nullable distribution ---
is_nullable
YES    1615
Name: count, dtype: int64


## 3. Quality Checks

In [52]:
# Check for Fivetran metadata columns (these are sync artifacts, not business data)
fivetran_cols = df[df["column_name"].str.startswith("_fivetran")]
print(f"Fivetran metadata rows: {len(fivetran_cols)} ({len(fivetran_cols)/len(df)*100:.1f}%)")
print(f"Fivetran column names: {fivetran_cols['column_name'].unique().tolist()}")
print(f"Tables with fivetran cols: {fivetran_cols['table_name'].nunique()}")

# Check for duplicate (table_name, column_name) pairs
dupes = df.duplicated(subset=["table_name", "column_name"], keep=False)
print(f"\nDuplicate (table, column) pairs: {dupes.sum()}")
if dupes.sum() > 0:
    print(df[dupes].sort_values(["table_name", "column_name"]))

Fivetran metadata rows: 270 (16.7%)
Fivetran column names: ['_fivetran_synced', '_fivetran_deleted', '_fivetran_id']
Tables with fivetran cols: 145

Duplicate (table, column) pairs: 0


In [53]:
# Check for whitespace issues in string columns
for col in ["table_name", "column_name", "data_type"]:
    leading = df[col].str.startswith(" ").sum()
    trailing = df[col].str.endswith(" ").sum()
    empty = (df[col].str.strip() == "").sum()
    print(f"{col}: leading_ws={leading}, trailing_ws={trailing}, empty_strings={empty}")

# Check for naming inconsistencies
print(f"\nSample table names:\n{sorted(df['table_name'].unique())[:20]}")

table_name: leading_ws=0, trailing_ws=0, empty_strings=0
column_name: leading_ws=0, trailing_ws=0, empty_strings=0
data_type: leading_ws=0, trailing_ws=0, empty_strings=0

Sample table names:
['applied_coupon', 'applied_gift_certificate', 'available_filter', 'banner', 'brand', 'brand_meta_keyword', 'brand_metafield', 'cart', 'cart_coupon', 'cart_custom_item', 'cart_digital_item', 'cart_discount', 'cart_gift_certificate_item', 'cart_physical_item', 'catalog_variant', 'catalog_variant_metafield', 'catalog_variant_option_value', 'category_metafield', 'category_setting', 'category_tree']


## 3b. Deeper Noise Analysis

Check for tables/columns that will confuse the text2sql model — things a business analyst would never ask about.

In [54]:
# Categorize tables by business relevance for text2sql
# The model sees "table.column (type)" — noisy tables pollute retrieval

all_tables = sorted(df_clean["table_name"].unique())

# Tables that are store/CMS configuration — not analytical
config_tables = [t for t in all_tables if any(t.endswith(s) for s in ["_setting", "_settings"]) 
                 or t in [
    "robot_setting", "locale_setting", "status_setting", "seo_setting",
    "security_setting", "search_setting", "email_setting",
    "customer_setting", "customer_setting_per_channel",
    "storefront_product_setting", "category_setting",
    "store_profile_setting",
]]

# Theme/widget/placement — CMS frontend, not data
cms_tables = [t for t in all_tables if t.startswith(("theme", "widget", "placement"))]

# Metafield tables — generic key-value stores, model can't query without knowing keys
metafield_tables = [t for t in all_tables if "metafield" in t]

# Form field tables — JSON blobs, not queryable in SQL
form_field_tables = [t for t in all_tables if "form_field" in t]

# Consent tables — GDPR compliance, not analytical
consent_tables = [t for t in all_tables if "consent" in t]

# Email template tables
email_tables = [t for t in all_tables if t.startswith("email_template")]

# Blog tables — content, not ecommerce analytics  
blog_tables = [t for t in all_tables if "blog" in t]

# Site routing/redirect — infrastructure
infra_tables = [t for t in all_tables if t in [
    "redirect", "site_route", "site_url", "custom_template_association",
    "robot_setting",
]]

# Filter config tables — storefront UI
filter_tables = [t for t in all_tables if t in [
    "available_filter", "enabled_filter", "contextual_filter",
]]

noisy_tables = set(config_tables + cms_tables + metafield_tables + form_field_tables +
                   consent_tables + email_tables + blog_tables + infra_tables + filter_tables)

noisy_rows = df_clean[df_clean["table_name"].isin(noisy_tables)]
print(f"Noisy tables: {len(noisy_tables)} ({len(noisy_rows)} rows)")
print(f"\n--- By category ---")
for label, tables in [
    ("Config/Settings", config_tables),
    ("CMS (theme/widget/placement)", cms_tables),
    ("Metafield (key-value)", metafield_tables),
    ("Form fields (JSON)", form_field_tables),
    ("Consent (GDPR)", consent_tables),
    ("Email templates", email_tables),
    ("Blog", blog_tables),
    ("Infrastructure", infra_tables),
    ("Filter config", filter_tables),
]:
    if tables:
        cols = df_clean[df_clean["table_name"].isin(tables)].shape[0]
        print(f"  {label}: {len(tables)} tables, {cols} cols → {tables}")

Noisy tables: 0 (0 rows)

--- By category ---


In [55]:
# Check noisy COLUMNS within otherwise useful tables
# These are columns that will confuse retrieval even in good tables

# 1. Product open_graph columns (SEO social media tags)
og_cols = df_clean[df_clean["column_name"].str.startswith("open_graph")]
print(f"Open Graph (SEO) columns: {len(og_cols)}")

# 2. product_variant_option config_* columns (form UI config, not data)
config_cols = df_clean[(df_clean["table_name"] == "product_variant_option") & 
                        df_clean["column_name"].str.startswith("config_")]
print(f"product_variant_option config_* cols: {len(config_cols)}")

# 3. Payment card security / PCI columns in order_transaction
pci_cols = {"avs_result_code", "avs_result_message", "avs_result_postal_match",
            "avs_result_street_match", "cvv_result_code", "cvv_result_message",
            "card_iin", "card_last_4", "card_expiry_month", "card_expiry_year",
            "payment_instrument_token", "gateway_transaction_id", "fraud_review",
            "test"}
pci_rows = df_clean[df_clean["column_name"].isin(pci_cols)]
print(f"PCI/card security columns: {len(pci_rows)}")

# 4. UUID/token internal identifiers
uuid_cols = df_clean[df_clean["column_name"].str.contains("uuid|token", case=False) & 
                      ~df_clean["column_name"].isin({"currency_code", "token"})]
print(f"UUID/token internal IDs: {len(uuid_cols)}")

# 5. Layout/template file references
layout_cols = df_clean[df_clean["column_name"].isin({"layout_file", "template_file"})]
print(f"Layout/template file refs: {len(layout_cols)}")

# 6. URL columns in non-product tables (secure_url, control_panel_base_url, etc.)
url_cols = df_clean[df_clean["column_name"].str.contains("_url$|^url$", regex=True) &
                     ~df_clean["column_name"].isin({"custom_url"})]
print(f"URL columns remaining: {len(url_cols)}")
print(url_cols[["table_name", "column_name"]].to_string(index=False))

total_noisy = len(noisy_rows) + len(og_cols) + len(config_cols) + len(pci_rows) + len(uuid_cols) + len(layout_cols) + len(url_cols)
print(f"\n=== Total noise: ~{total_noisy} rows that could confuse the model ===")

Open Graph (SEO) columns: 0
product_variant_option config_* cols: 0
PCI/card security columns: 0
UUID/token internal IDs: 0
Layout/template file refs: 0
URL columns remaining: 0
Empty DataFrame
Columns: [table_name, column_name]
Index: []

=== Total noise: ~0 rows that could confuse the model ===


## 3c. Business Relevance — Core vs. Noise

Categorize all 104 remaining tables by how likely a business analyst would query them.

In [59]:
# What's a business analyst actually going to ask?
# "Total revenue last month?", "Top products by sales?", "Orders by country?",
# "Customer retention?", "Which coupon performed best?"
#
# CORE: Tables that directly answer business questions
# SECONDARY: Useful for JOINs but rarely queried directly  
# NOISE: Analyst would never ask about these

remaining = sorted(df_clean["table_name"].unique())

core_tables = {
    # Orders & revenue (the #1 use case)
    "orders", "order_product", "order_coupon", "order_refund",
    "order_shipping_address", "order_shipment", "order_tax",
    "order_transaction", "order_status",
    # Products & catalog
    "product", "product_category", "product_variant", "product_image",
    "product_review", "product_custom_field",
    "brand", "category_tree",
    # Customers
    "customer", "customer_address", "customer_group",
    # Coupons & discounts (analysts do ask "which coupon performed best?")
    "coupon", "order_coupon",
    # Channels
    "channel", "channel_listing",
    # Currency & geography
    "currency", "country", "country_state",
    # Shipping
    "shipping_zone", "shipping_zone_location", "shipping_zone_method",
    # Pricing
    "price_list", "price_list_record", "price_list_assignment",
    # Gift certificates
    "gift_certificate",
    # Subscribers
    "subscriber",
    # Cart (for conversion / abandonment analysis)
    "cart",
    # Tax
    "tax_class",
    # Wishlist
    "wishlist", "wishlist_item",
    # Store info
    "store",
}

secondary_tables = remaining_set = set(remaining) - core_tables

core_rows = df_clean[df_clean["table_name"].isin(core_tables)]
secondary_rows = df_clean[~df_clean["table_name"].isin(core_tables)]

print(f"CORE tables: {len(core_tables & set(remaining))} tables, {len(core_rows)} columns")
print(f"SECONDARY/NOISE: {len(secondary_tables)} tables, {len(secondary_rows)} columns")
print(f"\n--- Tables NOT in core (candidates for removal) ---")
for t in sorted(secondary_tables):
    n = df_clean[df_clean["table_name"] == t].shape[0]
    print(f"  {t} ({n} cols)")
    
print(f"\n--- Core tables ---")
for t in sorted(core_tables & set(remaining)):
    n = df_clean[df_clean["table_name"] == t].shape[0]
    print(f"  {t} ({n} cols)")

CORE tables: 39 tables, 569 columns
SECONDARY/NOISE: 65 tables, 418 columns

--- Tables NOT in core (candidates for removal) ---
  applied_coupon (6 cols)
  applied_gift_certificate (6 cols)
  banner (11 cols)
  brand_meta_keyword (2 cols)
  cart_coupon (5 cols)
  cart_custom_item (7 cols)
  cart_digital_item (16 cols)
  cart_discount (3 cols)
  cart_gift_certificate_item (10 cols)
  cart_physical_item (20 cols)
  catalog_variant (21 cols)
  catalog_variant_option_value (5 cols)
  category_tree_meta_keyword (2 cols)
  channel_active_theme (1 cols)
  channel_app (2 cols)
  channel_currency_assignment (2 cols)
  channel_enabled_currency (2 cols)
  channel_variant (10 cols)
  checkout (28 cols)
  checkout_billing_address_custom_field (1 cols)
  consignment (22 cols)
  consignment_billing_address_custom_field (1 cols)
  consignment_coupon_discount (3 cols)
  consignment_discount (2 cols)
  consignment_line_item (2 cols)
  consignment_shipping_option (7 cols)
  coupon_country_restriction (2

## 4. Cleaning

**Strategy:** Whitelist only the 39 core business tables, then clean noisy columns within those.

**Core tables kept:**
- Orders & revenue: `orders`, `order_product`, `order_coupon`, `order_refund`, `order_shipping_address`, `order_shipment`, `order_tax`, `order_transaction`, `order_status`
- Products: `product`, `product_category`, `product_variant`, `product_image`, `product_review`, `product_custom_field`, `brand`, `category_tree`
- Customers: `customer`, `customer_address`, `customer_group`
- Coupons: `coupon`, `order_coupon`
- Channels: `channel`, `channel_listing`
- Geography: `currency`, `country`, `country_state`
- Shipping: `shipping_zone`, `shipping_zone_location`, `shipping_zone_method`
- Pricing: `price_list`, `price_list_record`, `price_list_assignment`
- Other: `gift_certificate`, `subscriber`, `cart`, `tax_class`, `wishlist`, `wishlist_item`, `store`

**Column-level removals within core tables:**
- Fivetran metadata, eBay, IP addresses, image URLs
- Duplicate checkout columns, API resource links
- Open Graph SEO, PCI/card security, UUIDs/tokens
- Layout files, template blobs, remaining URLs

In [60]:
df_clean = df.copy()

# 1. Strip whitespace
for col in ["table_name", "column_name", "data_type"]:
    df_clean[col] = df_clean[col].str.strip()

# ============================================================
# WHITELIST: Keep only core business tables
# ============================================================
core_tables = {
    # Orders & revenue
    "orders", "order_product", "order_coupon", "order_refund",
    "order_shipping_address", "order_shipment", "order_tax",
    "order_transaction", "order_status",
    # Products & catalog
    "product", "product_category", "product_variant", "product_image",
    "product_review", "product_custom_field",
    "brand", "category_tree",
    # Customers
    "customer", "customer_address", "customer_group",
    # Coupons
    "coupon",
    # Channels
    "channel", "channel_listing",
    # Currency & geography
    "currency", "country", "country_state",
    # Shipping
    "shipping_zone", "shipping_zone_location", "shipping_zone_method",
    # Pricing
    "price_list", "price_list_record", "price_list_assignment",
    # Other business tables
    "gift_certificate", "subscriber", "cart", "tax_class",
    "wishlist", "wishlist_item", "store",
}

before = len(df_clean)
df_clean = df_clean[df_clean["table_name"].isin(core_tables)]
print(f"[1] Kept {len(core_tables)} core tables → removed {before - len(df_clean)} rows from non-core tables")

# ============================================================
# COLUMN-LEVEL REMOVALS within core tables
# ============================================================

# 2. Remove Fivetran metadata
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].str.startswith("_fivetran")]
print(f"[2] Removed {before - len(df_clean)} Fivetran metadata rows")

# 3. Remove eBay integration columns
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].str.startswith("ebay_")]
print(f"[3] Removed {before - len(df_clean)} eBay integration columns")

# 4. Remove IP address columns (PII)
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].isin({"ip_address", "ip_address_v_6", "registration_ip_address"})]
print(f"[4] Removed {before - len(df_clean)} IP address columns")

# 5. Remove image/CDN URL columns
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].isin({
    "image_url", "thumbnail_url", "tiny_url", "standard_url",
    "logo_url", "icon_url", "image_file", "thumbnail_path", "icon_name"
})]
print(f"[5] Removed {before - len(df_clean)} image/CDN URL columns")

# 6. Remove API resource/URL navigation columns
api_cols = {
    ("orders", "coupon_resource"), ("orders", "coupon_url"),
    ("orders", "product_resource"), ("orders", "product_url"),
    ("orders", "shipping_addressresource"), ("orders", "shipping_addressurl"),
    ("order_shipping_address", "shipping_quotes_resource"),
    ("order_shipping_address", "shipping_quotes_url"),
    ("country", "state_resource"), ("country", "state_url"),
}
before = len(df_clean)
mask = df_clean.apply(lambda r: (r["table_name"], r["column_name"]) in api_cols, axis=1)
df_clean = df_clean[~mask]
print(f"[6] Removed {before - len(df_clean)} API resource/URL columns")

# 7. Open Graph SEO columns on product
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].str.startswith("open_graph")]
print(f"[7] Removed {before - len(df_clean)} Open Graph SEO columns")

# 8. PCI / card security columns
pci_cols = {
    "avs_result_code", "avs_result_message", "avs_result_postal_match",
    "avs_result_street_match", "cvv_result_code", "cvv_result_message",
    "card_iin", "card_last_4", "card_expiry_month", "card_expiry_year",
    "payment_instrument_token", "gateway_transaction_id", "fraud_review", "test",
}
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].isin(pci_cols)]
print(f"[8] Removed {before - len(df_clean)} PCI/card security columns")

# 9. UUID/token internal identifiers
before = len(df_clean)
df_clean = df_clean[~(df_clean["column_name"].str.contains("uuid|token", case=False)
                       & ~df_clean["column_name"].isin({"currency_code"}))]
print(f"[9] Removed {before - len(df_clean)} UUID/token columns")

# 10. Layout/template file references
before = len(df_clean)
df_clean = df_clean[~df_clean["column_name"].isin({"layout_file", "template_file"})]
print(f"[10] Removed {before - len(df_clean)} layout/template file columns")

# 11. Remaining non-data URL columns
url_remove = {
    ("store", "control_panel_base_url"), ("store", "secure_url"),
}
before = len(df_clean)
mask = df_clean.apply(lambda r: (r["table_name"], r["column_name"]) in url_remove, axis=1)
df_clean = df_clean[~mask]
print(f"[11] Removed {before - len(df_clean)} remaining URL columns")

# ============================================================
# FINAL STEPS
# ============================================================

# 12. Drop exact duplicates
before = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=["table_name", "column_name", "data_type"])
print(f"[12] Removed {before - len(df_clean)} duplicate rows")

# 13. Keep only needed columns & sort
df_clean = df_clean[["table_name", "column_name", "data_type"]]
df_clean = df_clean.sort_values(["table_name", "column_name"]).reset_index(drop=True)

print(f"\n{'='*50}")
print(f"Cleaned: {df.shape[0]} → {df_clean.shape[0]} rows ({df.shape[0] - df_clean.shape[0]} removed)")
print(f"Tables:  {df['table_name'].nunique()} → {df_clean['table_name'].nunique()}")
df_clean.head(10)

[1] Kept 39 core tables → removed 913 rows from non-core tables
[2] Removed 76 Fivetran metadata rows
[3] Removed 3 eBay integration columns
[4] Removed 3 IP address columns
[5] Removed 8 image/CDN URL columns
[6] Removed 10 API resource/URL columns
[7] Removed 6 Open Graph SEO columns
[8] Removed 14 PCI/card security columns
[9] Removed 9 UUID/token columns
[10] Removed 2 layout/template file columns
[11] Removed 2 remaining URL columns
[12] Removed 0 duplicate rows

Cleaned: 1615 → 569 rows (1046 removed)
Tables:  145 → 39


,table_name,column_name,data_type
0,brand,custom_url,STRING
1,brand,custom_url_is_customized,BOOL
2,brand,id,INT64
3,brand,meta_description,STRING
4,brand,name,STRING
5,brand,page_title,STRING
6,brand,search_keywords,STRING
7,cart,base_amount,FLOAT64
8,cart,cart_amount,FLOAT64
9,cart,channel_id,INT64


## 5. Summary per Table

In [61]:
# Cleaned schema overview: columns per table and data types used
summary = (
    df_clean.groupby("table_name")
    .agg(
        num_columns=("column_name", "count"),
        data_types=("data_type", lambda x: ", ".join(sorted(x.unique())))
    )
    .sort_values("num_columns", ascending=False)
)
summary

,num_columns,data_types
table_name,,
orders,72,"BOOL, FLOAT64, INT64, STRING, TIMESTAMP"
product,59,"BOOL, FLOAT64, INT64, STRING, TIMESTAMP"
store,44,"BOOL, INT64, STRING"
order_product,43,"BOOL, FLOAT64, INT64, STRING, TIMESTAMP"
order_shipping_address,29,"FLOAT64, INT64, STRING"
product_variant,23,"BOOL, FLOAT64, INT64, STRING"
order_shipment,23,"INT64, STRING, TIMESTAMP"
order_transaction,20,"FLOAT64, INT64, STRING, TIMESTAMP"
customer,16,"BOOL, INT64, STRING, TIMESTAMP"


## 6. Export Cleaned Schema

In [62]:
output_path = "../schemas/bigcommerce_schema_cleaned.csv"
df_clean.to_csv(output_path, index=False)
print(f"Saved cleaned schema to {output_path}")
print(f"  {len(df_clean)} rows, {df_clean['table_name'].nunique()} tables")

# Verify round-trip
df_verify = pd.read_csv(output_path)
assert list(df_verify.columns) == ["table_name", "column_name", "data_type"]
assert len(df_verify) == len(df_clean)
print("  Round-trip verification passed ✓")

Saved cleaned schema to ../schemas/bigcommerce_schema_cleaned.csv
  569 rows, 39 tables
  Round-trip verification passed ✓
